# doc2vec

In [7]:
import jieba 
import gensim
from gensim.models import doc2vec

from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

def m_doc(doclist):
    reslist = []
    for i, doc in enumerate(doclist):
        reslist.append(doc2vec.TaggedDocument(doc, [i]))
    return reslist

corp = m_doc(df2.fenci)
corp[:2]

d2vmodel = gensim.models.Doc2Vec(vector_size = 300, window = 5, min_count = 5)
%time d2vmodel.build_vocab(corp) #52s
d2vmodel.wv.vocab #dict

# 将新文本转换为相应维度空间下的向量
newvec = d2vmodel.infer_vector(df2.fenci[0])
print(df2.filename[0])
simfile = d2vmodel.docvecs.most_similar([newvec], topn = 10)
for i in range(len(simfile)):
    print(simfile[i])
    a,b = simfile[i]
    print(df.filename[a])

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

# 结果不符合预期

# 1.gensim实现文本相似度----原始词频LDA

In [1]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist]

from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 5, passes = 10) #拟合LDA模型
print(ldamodel.print_topics(num_topics = 5,num_words = 10))
from sklearn.externals import joblib
joblib.dump(ldamodel,'ldamodel_0506_5.model')

# 计算各语料的LDA模型值
corpus_lda = ldamodel[corpus] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)

from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus) # 使用的矩阵种类需要和拟合模型时相同
simmtx.index[:2]

# 使用gensim的LDA拟合结果进行演示
query_bow = dictionary.doc2bow(df2.fenci[20])
print(df2.filename[20])
lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

for i in range(len(sims)):
    print(sims[i])
    a,b = sims[i]
    if b != 0.0:
        print(df.filename[a])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.084 seconds.
Prefix dict has been built succesfully.
C:\Tools\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 3.58 s
[(0, '0.015*"创业" + 0.010*"大学生" + 0.008*"创新" + 0.008*"教育" + 0.008*"算法" + 0.007*"模型" + 0.006*"样本" + 0.005*"高校" + 0.005*"方法" + 0.005*"研究"'), (1, '0.010*"农业" + 0.007*"贫困" + 0.007*"耕地" + 0.005*"发展" + 0.005*"改革" + 0.005*"水价" + 0.004*"分析" + 0.004*"休闲" + 0.004*"巢湖" + 0.004*"利用"'), (2, '0.015*"数据" + 0.009*"密封胶" + 0.007*"医疗" + 0.006*"项目" + 0.005*"研究" + 0.005*"技术" + 0.005*"硅烷" + 0.005*"公立医院" + 0.005*"工程造价" + 0.005*"改性"'), (3, '0.015*"发展" + 0.011*"教学" + 0.011*"农村" + 0.010*"金融" + 0.009*"学生" + 0.008*"施工" + 0.008*"农业" + 0.007*"建筑" + 0.007*"专业" + 0.006*"工程"'), (4, '0.018*"医院" + 0.015*"医疗" + 0.015*"检查" + 0.010*"患者" + 0.008*"互认" + 0.008*"服务" + 0.005*"新生儿" + 0.005*"影响" + 0.005*"分析" + 0.005*"文本"')]
[(0, 0.9509325), (4, 0.048740093)]
[(3, 0.99892944)]
[(4, 0.99951422)]
[(4, 0.99945271)]
[(1, 0.95187932), (3, 0.047730125)]
[(4, 0.99949998)]
[(0, 0.46289358), (3, 0.50409997), (4, 0.032353625)]
[(3, 0.99873185)]
[(1, 0.99960452)]
[(3, 0.99964827)]
[(4, 0.99935549)]
[(1, 0.70819765), (3, 0.29

# 结果还可以

# 2.gensim实现文本相似度----TF-IDF

In [33]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist]
tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf
from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus_tfidf, id2word = dictionary, num_topics = 10, passes = 10) #拟合LDA模型
print(ldamodel.print_topics(num_topics = 10,num_words = 10))

# 计算各语料的LDA模型值
corpus_lda = ldamodel[corpus_tfidf] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)

from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus_tfidf) # 使用的矩阵种类需要和拟合模型时相同
simmtx.index[:2]

# 使用gensim的LDA拟合结果进行演示
query_bow = dictionary.doc2bow(df2.fenci[40])
print(df2.filename[40])
query_tfidf = tfidf_model[query_bow]
lda_vec = ldamodel[query_tfidf] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

for i in range(len(sims)):
    print(sims[i])
    a,b = sims[i]
    if b != 0.0:
        print(df.filename[a])

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

# 结果不好

# sklearn实现文本相似度--原始词频

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

import jieba
from sklearn.feature_extraction.text import CountVectorizer
#先弄出文档词频矩阵，然后进行相应的模型训练
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
rawfile=[]#空格分隔开的文本
for i in range(len(df2)):
    rawfile.append(' '.join(df2['fenci'][i]))
#print(rawfile) 
#rawfile listoflist ,每一项为用空格连接的一篇文档
countvec = CountVectorizer(min_df = 0.1)#最小词频
X = countvec.fit_transform(rawfile)#稀疏矩阵


from sklearn.metrics.pairwise import pairwise_distances

sims = pairwise_distances(X, metric = 'cosine')
print(sims.shape)  #50*50的矩阵
print(sims)


['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

# 文本检索

In [10]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
filelist=[]
for i in range(len(df2)):
    filelist.append(df2['fenci'][i])
#生成文档对应的字典和bow稀疏矩阵
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist]

from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus, id2word = dictionary, num_topics = 10, passes = 10) #拟合LDA模型
print(ldamodel.print_topics(num_topics = 10,num_words = 10))

# 计算各语料的LDA模型值
corpus_lda = ldamodel[corpus] # 此处应当使用和模型训练时相同类型的矩阵
for doc in corpus_lda:
    print(doc)

from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus) # 使用的矩阵种类需要和拟合模型时相同
simmtx.index[:2]

# 使用gensim的LDA拟合结果进行演示

#输入语句“电力文本数据挖掘”
import jieba
str = '医疗机构医院患者'
stoplist = list(pd.read_csv('C:/TMdata/stopwords.txt',names=['w'],sep='aaa',encoding='UTF-8',engine='python').w)
res = [w for w in jieba.cut_for_search(str) if w not in stoplist and len(w) > 1 ]

query_bow = dictionary.doc2bow(res)
lda_vec = ldamodel[query_bow] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

for i in range(len(sims)):
    print(sims[i])
    a,b = sims[i]
    if b != 0.0:
        print(df.filename[a])

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数